<a href="https://colab.research.google.com/github/jithesh9539/Approximate_Nearest-Neighbor_Search/blob/main/ANN_Algotithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install lightfm
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
import pickle
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
import numpy as np

In [5]:
from lightfm.datasets import fetch_movielens
movielens = fetch_movielens()

In [6]:
train = movielens['train']
test = movielens['test']

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.59, test 0.10.
AUC: train 0.89, test 0.85.


In [7]:

item_vectors = movielens['item_features'] * model.item_embeddings

In [8]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [9]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.60, test 0.11.
AUC: train 0.94, test 0.90.


In [10]:
with open('movielens.pickle', 'wb') as f:
    pickle.dump({"name": movielens['item_feature_labels'], "vector": item_vectors}, f)

In [11]:
def load_data():
    with open('movielens.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[ 0.23409992, -0.649664  ,  0.32381263, ...,  0.2666248 ,
         -0.58260924,  0.257941  ],
        [ 0.02928501, -0.15290615,  0.01266694, ..., -0.53318506,
         -0.53478944,  0.08389454],
        [ 0.3817396 , -0.4372891 ,  0.39676225, ...,  0.29956925,
         -0.30343893,  0.42843935],
        ...,
        [ 0.10886277, -0.10511472, -0.00861692, ...,  0.04481719,
          0.2430882 , -0.03099445],
        [ 0.06991807, -0.09576913, -0.01252945, ..., -0.02118149,
         -0.06926748, -0.05114214],
        [ 0.02546378, -0.01588266,  0.09501867, ...,  0.04091675,
         -0.07064592,  0.09796126]], dtype=float32)}

In [12]:
!pip install faiss-gpu
import faiss

**Locality_Sensitive_Hashing**

In [13]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
          
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [14]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [15]:
movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movie_questions}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Lion King, The (1994)
* Star Trek III: The Search for Spock (1984)
* Star Trek: The Wrath of Khan (1982)
* Nightmare Before Christmas, The (1993)
* Maverick (1994)
* Man Without a Face, The (1993)
* Robin Hood: Prince of Thieves (1991)
* Nell (1994)
* Aladdin (1992)
* Sleepless in Seattle (1993)


**Trees** and Graphs using Annoy

In [16]:
class TreesIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [17]:
!pip install annoy
import annoy
index = TreesIndex(data["vector"], data["name"])
index.build()

     |████████████████████████████████| 646 kB 5.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391672 sha256=84e406f029e41c5e7ce8de6e3412571af07d987c1281e0005e48554f2070bd4e
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  if __name__ == '__main__':


In [18]:
!pip install annoy
import annoy

In [19]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
similar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

The most similar movie to Nightmare Before Christmas, The (1993) are:
* Lion King, The (1994)
* Star Trek III: The Search for Spock (1984)
* Star Trek: The Wrath of Khan (1982)
* Nightmare Before Christmas, The (1993)
* Maverick (1994)
* Man Without a Face, The (1993)
* Robin Hood: Prince of Thieves (1991)
* Nell (1994)
* Aladdin (1992)
* Sleepless in Seattle (1993)


In [20]:
!pip install nmslib
import nmslib

     |████████████████████████████████| 13.5 MB 5.2 MB/s 
     |████████████████████████████████| 188 kB 49.7 MB/s 


**HNSW: Hierarchical Navigable Small World Algorithm**

In [21]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [22]:
index = HNSWIndex(data["vector"], data["name"])
index.build()

In [23]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar stack to {movie_name} are:\n* {simlar_movie_questions}")

The most similar stack to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Star Trek: Generations (1994)
* Star Trek III: The Search for Spock (1984)
* Star Trek: The Wrath of Khan (1982)
* Conan the Barbarian (1981)
* Star Trek IV: The Voyage Home (1986)
* Pretty Woman (1990)
* Mrs. Doubtfire (1993)
* Maverick (1994)
* Madame Butterfly (1995)


**Product** **Quantization**

In [24]:
class ProductIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimension)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimension, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [25]:
index = ProductIndex(data["vector"], data["name"])
index.build()

In [26]:
movie_index = 90
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movie to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movie to Nightmare Before Christmas, The (1993) are:


['Nightmare Before Christmas, The (1993)',
 'Sleepless in Seattle (1993)',
 'Hunt for Red October, The (1990)',
 'Sound of Music, The (1965)',
 'Groundhog Day (1993)',
 'Princess Bride, The (1987)',
 'Sneakers (1992)',
 'Dave (1993)',
 'Speed (1994)',
 'True Lies (1994)']

**Exhaustive** **Search**

In [27]:
class ExhaustiveIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [28]:
index = ExhaustiveIndex(data["vector"], data["name"])
index.build()

In [29]:
movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movie_questions = '\n* '.join(index.query(movie_vector))
print(f"The most similar movie to {movie_name} are:\n* {simlar_movie_questions}")

The most similar movie to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Star Trek: Generations (1994)
* Conan the Barbarian (1981)
* Maverick (1994)
* Pretty Woman (1990)
* Heavy Metal (1981)
* Sleepless in Seattle (1993)
* Lion King, The (1994)
* Star Trek III: The Search for Spock (1984)
* Groundhog Day (1993)
